In [1]:
from dsview.content_loader import UrlLoader

In [13]:
from langchain_openai import ChatOpenAI

In [14]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [65]:
url_loader = UrlLoader("http://www.anthropic.com/news/claude-3-5-sonnet")

In [23]:
url_loader = UrlLoader("https://www.rungalileo.io/hallucinationindex")

In [66]:
url_loader.load()

/Users/tdurouchoux/Documents/projects/dsview/dsview/content_loader.py:52: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 52 of the file /Users/tdurouchoux/Documents/projects/dsview/dsview/content_loader.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  self.content_soup = BeautifulSoup(response.content)


In [32]:
links = url_loader.content_soup.find_all("a")

In [67]:
links_list = []

links = url_loader.content_soup.find_all("a")
for link in links:
    link_url = link.get("href")
    if link_url is not None and link_url.startswith("http"):
        url = link.get("href")
        print("Link:", link.get("href"), "Text:", link.string)
        links_list.append(str(link))

Link: https://claude.ai/ Text: Try on Claude.ai
Link: https://aws.amazon.com/blogs/aws/anthropics-claude-3-5-sonnet-model-now-available-in-amazon-bedrock-the-most-intelligent-claude-model-yet/ Text: Amazon Bedrock
Link: https://cloud.google.com/blog/products/ai-machine-learning/announcing-anthropics-claude-3-5-sonnet-on-vertex-ai-providing-more-choice-for-enterprises Text: Google Cloud’s Vertex AI
Link: https://www-cdn.anthropic.com/fed9cc193a14b84131812372d8d5857f8f304c52/Model_Card_Claude_3_Addendum.pdf Text: internal agentic coding evaluation
Link: https://www.anthropic.com/news/tool-use-ga Text: provided with the relevant tools
Link: https://www.anthropic.com/news/anthropics-responsible-scaling-policy Text: ASL-2
Link: https://www-cdn.anthropic.com/fed9cc193a14b84131812372d8d5857f8f304c52/Model_Card_Claude_3_Addendum.pdf Text: model card addendum
Link: https://www.commerce.gov/news/press-releases/2024/04/us-and-uk-announce-partnership-science-ai-safety Text: announced earlier this 

In [62]:
links_list

['<a class="block py-4 text-sm duration-200 hover:opacity-80" href="https://github.com/rungalileo/hallucination-index" target="_blank">Github</a>',
 '<a class="duration-200 hover:opacity-80" href="https://github.com/rungalileo/hallucination-index" target="_blank">Github</a>',
 '<a class="my-8 text-[#5C5CFF]" href="https://docs.rungalileo.io/galileo/gen-ai-studio-products/guardrail-store/context-adherence/groundedness" target="_blank">Context Adherence</a>',
 '<a class="my-8 text-[#5C5CFF]" href="https://www.rungalileo.io/blog/chainpoll">ChainPoll</a>',
 '<a class="text-hi-100 hover:underline" href="https://www.together.ai/" rel="noopener noreferrer" target="_blank">Together.ai</a>',
 '<a class="text-hi-100 hover:underline" href="https://www.together.ai/" rel="noopener noreferrer" target="_blank">Together.ai</a>']

In [27]:
from langchain_core.prompts import ChatPromptTemplate

In [68]:
prompt = ChatPromptTemplate.from_template(
    """
You are part of a knowledge management system, your role is to help highlight the most relevant information from Data Science sources.
The source could be about mathematics, Python, Machine Learning, Large Language Model, Data engineering or any related subject.
You will be provided with the text of the source and links that are mentioned within the source. Your task is to select at most 5
links that are the most relevant technically relevant to the source. You must ignore any social networks links or internal website links, 
also you should not provide duplicate links.

The output should like this : 
1. [link 1 name](url 1) - quick description 1
2. [link 2 name](url 2) - quick description 2


Data Science source extracted from url {url}:
{input}

Links within this source : 
{links}
"""
)

In [69]:
link_selector = prompt | llm

In [70]:
link_selection = link_selector.invoke(
    {"url": url_loader.url, "input": url_loader.content, "links": "\n".join(links_list)})

In [71]:
print(link_selection.content)


1. [Amazon Bedrock](https://aws.amazon.com/blogs/aws/anthropics-claude-3-5-sonnet-model-now-available-in-amazon-bedrock-the-most-intelligent-claude-model-yet/) - Learn about the integration of Claude 3.5 Sonnet with Amazon Bedrock, enhancing AI capabilities for enterprises.
2. [Google Cloud’s Vertex AI](https://cloud.google.com/blog/products/ai-machine-learning/announcing-anthropics-claude-3-5-sonnet-on-vertex-ai-providing-more-choice-for-enterprises) - Discover how Claude 3.5 Sonnet is available on Google Cloud’s Vertex AI, providing more options for businesses.
3. [Model Card Addendum](https://www-cdn.anthropic.com/fed9cc193a14b84131812372d8d5857f8f304c52/Model_Card_Claude_3_Addendum.pdf) - Access the detailed model card addendum for Claude 3.5 Sonnet, which includes performance metrics and safety evaluations.
4. [ASL-2](https://www.anthropic.com/news/anthropics-responsible-scaling-policy) - Read about the ASL-2 safety classification and the measures taken to ensure the responsible u